In [1]:
#load packages
import sklearn  # collection of machine learning algorithms
from IPython import display  # pretty printing of dataframes in Jupyter notebook
# collection of functions for scientific computing and advance mathematics
import scipy as sp
import numpy as np  # foundational package for scientific computing
# collection of functions for scientific and publication-ready visualization
import matplotlib
import pandas as pd  # collection of functions for data processing and analysis modeled after R dataframes with SQL like features
from sklearn import metrics
from sklearn import model_selection
from sklearn import feature_selection
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import re as re
import io
import zipfile
from sklearn.model_selection import train_test_split
import requests
from sklearn.metrics import roc_curve, auc,log_loss
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import warnings
import time
import random
import IPython
import seaborn as sns
from hyperopt import hp
import xgboost as xgb
from pandas import Series, DataFrame
import sys  # access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))

print("SciPy version: {}". format(sp.__version__))

print("IPython version: {}". format(IPython.__version__))

print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries


#ignore warnings
warnings.filterwarnings('ignore')
print('-'*25)
# データインポートライブラリ

# データ加工・処理・分析ライブラリ


# 可視化ライブラリ
%matplotlib inline


#Model
%precision 3


SEED = 42


Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.22.4
SciPy version: 1.7.1
IPython version: 7.26.0
scikit-learn version: 0.24.2
-------------------------


# GBDT

In [2]:
GBDT_train = pd.read_csv("./df/EDA_09_GBDT_train.csv")
GBDT_test = pd.read_csv("./df/EDA_09_GBDT_test.csv")
Target = 'Perished'
GBDT_test.shape
data_cleaner = [GBDT_train, GBDT_test]

In [3]:
GBDT_test.isnull().sum()

PassengerId        0
Pclass             0
Name               0
Sex                0
Age               86
SibSp              0
Parch              0
Ticket             0
Fare               1
Cabin            327
Embarked           0
FamilySize         0
IsAlone            0
Title              0
FareBin            1
Sex_Code           0
Embarked_Code      0
Title_Code         0
FareBin_Code       0
AgeBin            86
Age_Code           0
dtype: int64

In [4]:
for dataset in data_cleaner:
    dataset = dataset.drop(columns=["Name", "Sex", "Age", "IsAlone", "Ticket", "SibSp", "Parch",
                           "FareBin_Code", "Cabin", "Embarked", "Title", "FareBin", "AgeBin"], inplace=True)


In [6]:
GBDT_train.columns

Index(['PassengerId', 'Perished', 'Pclass', 'Fare', 'FamilySize', 'Sex_Code',
       'Embarked_Code', 'Title_Code', 'Age_Code'],
      dtype='object')

In [11]:
USE_Columns = GBDT_train.columns
X = GBDT_train.iloc[:,2:].values
y = GBDT_train.iloc[:,1].values
X_test = GBDT_test.iloc[:,1:].values
# print('X_train shape: {}'.format(GBDT_X.shape))
# print('y_train shape: {}'.format(GBDT_y.shape))
# print('X_test shape: {}'.format(GBDT_X_test.shape))


In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.5, random_state=SEED)


In [16]:
#Model1 XGBoost
'''
Note
- Tuning はhyperoptで
- 決定木の数は十分大きくしてearly_stoppingにより抑制
- learning rate(eta)は学習時に0.1，提出時により小さくする．
- Turing時のKFoldはCrossValidateionにより1に，実際のモデル作成時にはSEEDにより分割
- uniform は一様分布から抽出する．
- quniformは一様分布から一定の間隔ごとに抽出する．
- logunigoromは対数が一様分布に従う分布から抽出する．


'''
params = {
    'booster':'gbtree',
    'objective':'binary:logistic',
    'eta':0.1,
    'gamma':0.0,
    'alpha':0.0,
    'lambda':1.0,
    'min_child_weight':1,
    'max_depth':5,
    'subsample':0.8,
    'colsample_bytree':0.8,
    'random_state':SEED
}
param_space = {
    'min_child_weight':hp.loguniform('min_child_weight',np.log(0.1),np.log(10)),
    'max_depth':hp.quniform('max_depth',3,9,1),
    'subsample':hp.quniform('subsample',0.6,0.95,0.05),
    'colsample_bytree':hp.quniform('colsample_bytree',0.6,0.95,0.05),
    'gamma':hp.loguniform('gamma',np.log(1e-8),np.log(1.0)),
    #余裕があれば
    # 'alpha':hp.loguniform('alpha',np.log(1e-8),np.log(1.0)),
    # 'lambda': hp.loguniform('alpha', np.log(1e-8), np.log(1.0)),

}
# XGBoodt 特徴量選択
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid,label= y_valid)
dtest = xgb.DMatrix(X_test)
watchlist = [(dtrain,'train'),(dvalid,'eval')]
num_round = 50



In [ ]:
def score(params):
    

In [14]:
model = xgb.train(params, dtrain, num_round, evals=watchlist)
va_pred = model.predict(dvalid)
score = log_loss(y_valid,va_pred)
print(score)


0.43366320370675604


In [15]:
pred = model.predict(dvalid)
pred

array([0.24 , 0.869, 0.776, 0.059, 0.287, 0.038, 0.194, 0.837, 0.173,
       0.051, 0.676, 0.957, 0.918, 0.679, 0.875, 0.059, 0.676, 0.342,
       0.687, 0.807, 0.878, 0.524, 0.652, 0.938, 0.936, 0.848, 0.407,
       0.83 , 0.848, 0.6  , 0.905, 0.286, 0.697, 0.45 , 0.937, 0.804,
       0.731, 0.194, 0.034, 0.925, 0.889, 0.951, 0.956, 0.895, 0.481,
       0.937, 0.937, 0.938, 0.933, 0.539, 0.174, 0.076, 0.94 , 0.489,
       0.9  , 0.089, 0.869, 0.06 , 0.044, 0.385, 0.854, 0.06 , 0.057,
       0.662, 0.895, 0.044, 0.852, 0.858, 0.559, 0.039, 0.078, 0.195,
       0.469, 0.051, 0.889, 0.923, 0.154, 0.044, 0.038, 0.366, 0.949,
       0.038, 0.072, 0.907, 0.624, 0.633, 0.041, 0.021, 0.664, 0.936,
       0.201, 0.853, 0.779, 0.914, 0.925, 0.851, 0.849, 0.957, 0.04 ,
       0.878, 0.644, 0.968, 0.047, 0.968, 0.932, 0.861, 0.061, 0.796,
       0.933, 0.669, 0.054, 0.884, 0.05 , 0.539, 0.598, 0.938, 0.617,
       0.761, 0.074, 0.806, 0.845, 0.032, 0.04 , 0.286, 0.869, 0.573,
       0.042, 0.576,

In [28]:
submission_ex = pd.read_csv("gender_submission.csv")
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = GBDT_test['PassengerId']
submission_df[Target] = XGB_pred
submission_df.reset_index(drop=True, inplace=True)
submission_df.to_csv('sub/09_GBDT_submissions.csv', header=True, index=False)
print("Submission Format:{} ".format(submission_ex.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df.head(20)


Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Perished
0,892,1
1,893,1
2,894,1
3,895,1
4,896,1
5,897,1
6,898,1
7,899,0
8,900,1
9,901,0


In [20]:
submission_df[Target].value_counts()

0    344
1     74
Name: Perished, dtype: int64